# Transformaciones en un RDD

Los RDDs son inmutables, y cada operación crea un nuevo RDD.
Las dos principales operaciones que se pueden realizar sobre un RDD son:
* **Transformaciones**: Las transformaciones cambian los elementos en el RDD (dividir el elemento de entrada, filtrar elementos o realizar cálculos de algún tipo). Varias transformaciones pueden realizarse en una secuencia, sin embargo, no se lleva a cabo ninguna ejecución durante la planificación. Spark agrega las transformaciones al DAG de cálculo (evaluación perezosa).
* **Acciones**

Las transformaciones se pueden dividir en 4 categorías:
* **Generales**: filter, distinct, union, map, flatMap, ...
* **Matemáticas y estadísticas**: mean, sum, max, min, variance, ...
* **De conjunto o relacionales**: join, intersection, subtract, distinct, groupByKey, ...
* **Basadas en estructuras de datos**: select, withColumn, drop, orderBy, groupBy, ...

# Función `map` de PySpark
La función `map` de PySpark es una operación de transformación que se aplica a un conjunto de datos distribuidos (RDD) en paralelo. Básicamente, la función `map` toma una función como entrada y aplica esta función a cada elemento del RDD, produciendo un nuevo RDD con los resultados.

Por ejemplo, si tenemos un RDD de números y queremos duplicar cada número, podemos usar la función `map` junto con una función que multiplique cada número por dos. De esta manera, PySpark aplicará esta función a cada elemento del RDD y devolverá un nuevo RDD con los números duplicados.

En resumen, la función `map` de PySpark es una herramienta útil para transformar un RDD en otro RDD con un formato diferente, mediante la aplicación de una función a cada elemento del RDD original.

In [2]:
import findspark

findspark.init()

from pyspark.sql import SparkSession
from pyspark import SparkContext

spark = SparkSession.builder.master("local[*]").appName("PySpark").getOrCreate()

sc: SparkContext = spark.sparkContext

In [0]:
rdd = sc.parallelize([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
rdd_sub = rdd.map(lambda x: x - 1).collect()
rdd_sub.collect()

In [3]:
rdd_pair = rdd.map(lambda x: x % 2 == 0)
rdd_pair.collect()

In [29]:
rdd_names = sc.parallelize(["John", "Fred", "Anna", "James", "Joe", "Joline", "Mary"])
rdd_names_upper = rdd_names.map(lambda x: x.upper())
rdd_names_upper.collect()

['JOHN', 'FRED', 'ANNA', 'JAMES', 'JOE', 'JOLINE', 'MARY']

In [8]:
rdd_hello = rdd_names.map(lambda x: "Hello " + x)
rdd_hello.collect()

['Hello John', 'Hello Fred', 'Hello Anna', 'Hello James']

# Función `flatMap` de PySpark
La función flatMap de PySpark es similar a la función map, ya que también se utiliza para transformar un RDD en otro RDD mediante la aplicación de una función a cada elemento del RDD. Sin embargo, la diferencia principal entre map y flatMap radica en el formato de salida del nuevo RDD.

Mientras que map aplica la función a cada elemento del RDD y devuelve un nuevo RDD con los resultados de la función, flatMap toma cada elemento del RDD y genera cero o más elementos para el nuevo RDD. En otras palabras, la función de flatMap devuelve una lista de elementos para cada elemento del RDD de entrada, y luego aplana estas listas en un solo RDD.

Por ejemplo, si tenemos un RDD de oraciones y queremos dividir cada oración en palabras, podemos usar la función flatMap junto con una función que divide cada oración en una lista de palabras. En este caso, flatMap generará una lista de palabras para cada oración y luego aplana todas estas listas en un solo RDD de palabras.

En resumen, la función flatMap de PySpark es una herramienta útil para transformar un RDD en otro RDD mediante la aplicación de una función a cada elemento del RDD, y generando cero o más elementos para el nuevo RDD.

In [0]:
rdd_squared = rdd.map(lambda x: (x, x ** 2))
rdd_squared.collect()

In [16]:
rdd_squared_flat = rdd.flatMap(lambda x: (x, x ** 2))
rdd_squared_flat.collect()

[1, 1, 2, 4, 3, 9, 4, 16, 5, 25]

In [18]:
rdd_names_lower_upper = rdd_names.map(lambda x: (x.lower(), x.upper()))
rdd_names_lower_upper.collect()

[('john', 'JOHN'), ('fred', 'FRED'), ('anna', 'ANNA'), ('james', 'JAMES')]

In [20]:
rdd_names_lower_upper_flat = rdd_names.flatMap(lambda x: (x.lower(), x.upper()))
rdd_names_lower_upper_flat.collect()

['john', 'JOHN', 'fred', 'FRED', 'anna', 'ANNA', 'james', 'JAMES']

# Función `filter` de PySpark
La función filter de PySpark se utiliza para filtrar elementos de un RDD basándose en una condición booleana. Es decir, toma como entrada una función que devuelve un valor booleano para cada elemento del RDD, y devuelve un nuevo RDD que solo contiene los elementos que cumplen la condición.

Por ejemplo, si tenemos un RDD de números y queremos filtrar solo los números pares, podemos usar la función filter junto con una función que compruebe si cada número es par o no. En este caso, filter recorrerá cada elemento del RDD y aplicará la función para comprobar si es par o no. Luego, devolverá un nuevo RDD que solo contiene los números pares.

En resumen, la función filter de PySpark es una herramienta útil para filtrar elementos de un RDD basándose en una condición booleana. Esta función devuelve un nuevo RDD que solo contiene los elementos que cumplen la condición especificada.

In [22]:
rdd_even = rdd.filter(lambda x: x % 2 == 0)
rdd_even.collect()

[2, 4]

In [23]:
rdd_odd = rdd.filter(lambda x: x % 2 != 0)
rdd_odd.collect()

[1, 3, 5]

In [24]:
# Just as reminder, this is the original RDD: rdd_names = sc.parallelize(["John", "Fred", "Anna", "James", "Joe", "Joline", "Mary"])
rdd_names_start_with_j = rdd_names.filter(lambda x: x.lower().startswith("j"))
rdd_names_start_with_j.collect()

['John', 'James']

In [30]:
rdd_names_start_with_j_and_contains_o = rdd_names.filter(
    lambda x: x.lower().startswith("j") and "o" in x.lower()
)
rdd_names_start_with_j_and_contains_o.collect()

['John', 'Joe', 'Joline']

# Función `coalesce` de PySpark
La función coalesce de PySpark se utiliza para reducir el número de particiones en un RDD. Es decir, toma como entrada el número de particiones que se desean en el nuevo RDD y devuelve un nuevo RDD que tiene ese número de particiones.

La función coalesce es útil cuando tenemos un RDD con muchas particiones y queremos reducir el número de particiones para mejorar el rendimiento de ciertas operaciones. Por ejemplo, si tenemos un RDD con 100 particiones y sabemos que solo necesitamos 10 particiones para realizar una operación determinada, podemos usar la función coalesce para reducir el número de particiones.

Es importante tener en cuenta que la función coalesce puede provocar una desigualdad en el tamaño de las particiones del nuevo RDD. En otras palabras, es posible que algunas particiones sean mucho más grandes que otras después de la reducción, lo que puede afectar el rendimiento de algunas operaciones.

En resumen, la función coalesce de PySpark se utiliza para reducir el número de particiones en un RDD y mejorar el rendimiento de ciertas operaciones.

In [31]:
# Just as reminder, this is the original RDD: rdd = sc.parallelize([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
rdd.getNumPartitions()

12

In [32]:
rdd_coalesce = rdd.coalesce(5)
rdd_coalesce.getNumPartitions()

5

# Funcion `repartition` de PySpark
La función repartition toma como entrada el número deseado de particiones y redistribuye los datos del RDD en esas particiones. Si el nuevo número de particiones es mayor que el número actual de particiones, se realizan operaciones de shuffle para redistribuir los datos de manera equilibrada. Por otro lado, si el nuevo número de particiones es menor que el número actual de particiones, la función repartition se comporta igual que la función coalesce.

Es importante tener en cuenta que la función repartition es una operación costosa, ya que implica una operación de shuffle que puede ser muy costosa en términos de tiempo y recursos. Por lo tanto, es importante utilizar esta función solo cuando sea necesario y asegurarse de ajustar el número de particiones al tamaño y la naturaleza de los datos del RDD.

En resumen, la función repartition de PySpark se utiliza para redistribuir los datos de un RDD en un número diferente de particiones, y puede ser una operación costosa en términos de tiempo y recursos.

In [33]:
rdd_coalesce.getNumPartitions()

5

In [34]:
rdd_repartition = rdd.repartition(10)
rdd_repartition.getNumPartitions()

10

# Función `reduceByKey` de PySpark
La función reduceByKey de PySpark es una operación de reducción que se utiliza para combinar los valores de un RDD que tienen la misma clave.

Por ejemplo, si tenemos un RDD con pares clave-valor (clave, valor) y queremos sumar los valores asociados a cada clave, podemos utilizar la función reduceByKey para realizar esta operación. La función reduceByKey combinará los valores de todos los pares con la misma clave, aplicando una función de reducción que se especifica como argumento, como la suma en este caso.

La función reduceByKey devuelve un nuevo RDD con pares clave-valor, donde las claves son las mismas que las del RDD original y los valores son el resultado de aplicar la función de reducción a los valores asociados a cada clave.

En resumen, la función reduceByKey de PySpark es una operación de reducción que se utiliza para combinar los valores de un RDD que tienen la misma clave, aplicando una función de reducción que se especifica como argumento.

In [38]:
rdd_base_reduce_by_key = sc.parallelize(
    [
        ("a", 1), ("b", 2), ("a", 3), ("b", 4), ("c", 5), ("a", 6),
        ("b", 7), ("c", 8), ("d", 9), ("a", 10), ("b", 11), ("c", 12)
    ]
)
# a = 20, b = 24, c = 25, d = 9
rdd_reduced = rdd_base_reduce_by_key.reduceByKey(lambda x, y: x + y)
rdd_reduced.collect()

[('b', 24), ('a', 20), ('d', 9), ('c', 25)]

[('b', 24), ('a', 20), ('d', 9), ('c', 25)]